# Preprocessing Data

In [126]:
import os
import sys
sys.path.append('../')

# import geoplot as gplt
import holidays
us_holidays = holidays.UnitedStates()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates
import math
import datetime
import rtree
import sys

import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon, Point
from pyproj import CRS

from setup import *

%matplotlib inline
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [326]:
period = 'after'
start_date = pd.to_datetime('20200302')
end_date = pd.to_datetime('20201031')

# period = 'before'
# start_date = pd.to_datetime('20190801')
# end_date = pd.to_datetime('20200301')


# Census Tract Shapefiles

In [128]:
# shapefile of census blocks 2010 
gblk = pd.read_csv(data_dir+'data_raw/CensusBlockTIGER2010.csv')
# craete a geometric object
gblk['the_geom'] = gblk['the_geom'].apply(wkt.loads)
# create the geo dataframe
block_gdf = gpd.GeoDataFrame(gblk, geometry='the_geom')
# Calculate the area of the blocks/polygons
block_gdf.crs = CRS('epsg:4326')
block_gdf = block_gdf.to_crs("epsg:26916")

In [177]:
tract_gdf = block_gdf.dissolve(by=['STATEFP10','COUNTYFP10','TRACTCE10'], 
                               as_index=False)[['STATEFP10','COUNTYFP10','TRACTCE10','the_geom']]
tract_gdf['area'] = tract_gdf['the_geom'].map(lambda p:p.area)
tract_gdf['GEOID10'] = tract_gdf['STATEFP10'].astype(str)+"_"+tract_gdf['COUNTYFP10'].astype(str)+"_"+tract_gdf['TRACTCE10'].astype(str)

In [162]:
print(len(tract_gdf), "census tracts.")

811 census tracts.


In [130]:
print('Census Tracts Columns: ', tract_gdf.columns.to_list())

Census Tracts Columns:  ['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'the_geom', 'area']


# Public Transit

In [327]:
d = pd.DataFrame()
df_iter = pd.read_csv(doe_data_dir+"raw_data/mobility/pt_aug19_oct20.csv", iterator=True, chunksize=1000000)
for df_chunk in df_iter:
    df_chunk['day'] = pd.to_datetime(df_chunk['day'])
    df_chunk = df_chunk[((df_chunk['day']>=start_date)&(df_chunk['day']<=end_date))]
    df_chunk['dow'] = df_chunk['dow'].str.strip()
    d = pd.concat((d, df_chunk), ignore_index=True)
    
d['day'] = pd.to_datetime(d['day']).dt.date
d['dow'] = d['dow'].str.strip()
print('Columns:', d.columns.tolist())

Columns: ['week', 'day', 'dow', 'quarter_hour', 'mode', 'stop_name', 'lat', 'lon', 'count']


In [328]:
# edit the temporal dimension.
days = d.groupby('day').size().reset_index().iloc[:, :-1].reset_index()
days['holiday'] = [int(date in us_holidays) for date in days['day']]
days.to_csv(project_data_dir+period+"_dates.csv", index=False)
daydict = pd.Series(days.index.values,index=days.day).to_dict()

In [329]:
print(days.iloc[0]['day'], days.iloc[-1]['day'])

2020-03-02 2020-10-31


In [239]:
d['day_index'] = d['day'].map(daydict)
# create ts as quarter_hour index
d['ts'] = (d['day_index'])*24*4 + d['quarter_hour']
d.dropna(inplace=True)

In [240]:
# rail counts wrt census tracts
d_points = d[['stop_name','mode','lat','lon']].drop_duplicates().dropna()
point_geom = [Point(xy) for xy in zip(d_points['lon'],d_points['lat'])]
crs = CRS("epsg:4326")
pointdf = gpd.GeoDataFrame(d_points, crs=crs, geometry=point_geom)
pointdf = pointdf.to_crs("epsg:26916")

stop_census = gpd.sjoin(tract_gdf, pointdf, how="inner", op='intersects')

d_census = d.merge(stop_census, on=['stop_name', 'mode', 'lat', 'lon'])

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [241]:
rail_df = d_census.loc[d_census['mode']=='CTA Rail']
rail_df = rail_df.sort_values(by=['GEOID10','day_index','quarter_hour']) 

In [242]:
rail_df = rail_df.groupby(['GEOID10', 'ts', 'day_index','day', 'dow', 'quarter_hour', 'mode'], as_index=False).sum()\
                [['GEOID10', 'ts', 'day_index', 'day', 'dow', 'quarter_hour', 'mode', 'count']]

In [243]:
bus_df = d_census.loc[d_census['mode']=='CTA Bus']
bus_df = bus_df.sort_values(by=['GEOID10','day_index','day','quarter_hour']) 

In [244]:
bus_df = bus_df.groupby(['GEOID10', 'ts', 'day_index', 'day', 'dow', 'quarter_hour', 'mode'], as_index=False).sum()\
                [['GEOID10', 'ts','day_index', 'day', 'dow', 'quarter_hour', 'mode', 'count']]
bus_df.columns = ['GEOID10', 'ts','day_index', 'day', 'dow', 'quarter_hour', 'mode', 'bus_count']

In [245]:
# save data
bus_df.to_csv(project_data_dir+'data_processed/census_tract/bus_df_'+start_date.strftime("%y%m%d")+'_'+end_date.strftime("%y%m%d")+'.csv', index=False)
rail_df.to_csv(project_data_dir+'data_processed/census_tract/rail_df_'+start_date.strftime("%y%m%d")+'_'+end_date.strftime("%y%m%d")+'.csv', index=False)

# TNC

In [330]:
tnc_files = [name[12:-4].split("_")+[name] for name in os.listdir(doe_data_dir+"processed_data/mobility/TNC") if name[:11]=='chicago_tnc']
tnc_files = pd.DataFrame(tnc_files, columns=['start_date','end_date','name'])
tnc_files['start_date'] = pd.to_datetime(tnc_files['start_date']).dt.date
tnc_files['end_date'] = pd.to_datetime(tnc_files['end_date']).dt.date
tnc_files.sort_values(by='start_date', inplace=True)

tnc = pd.DataFrame()
for s,e,name in zip(tnc_files['start_date'],tnc_files['end_date'],tnc_files['name']):        
    if (end_date >= s) | (start_date <= e):
        df_chunk = pd.read_csv(doe_data_dir+"processed_data/mobility/TNC/"+name)
        df_chunk['Trip Start Timestamp'] = pd.to_datetime(df_chunk['Trip Start Timestamp'])
        df_chunk = df_chunk.dropna(subset=['Pickup Census Tract'])
        if (end_date >= e) & (start_date <= s):
            print(name, 'all')
            tnc = tnc.append(df_chunk)
        else:
            print(name, 'part')
            tnc = tnc.append(df_chunk[(df_chunk['Trip Start Timestamp']>=start_date)&(df_chunk['Trip Start Timestamp']<=end_date)])


<ipython-input-330-2a617723eaa4>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= s) | (start_date <= e):
<ipython-input-330-2a617723eaa4>:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= e) & (start_date <= s):
<ipython-input-330-2a617723eaa4>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tnc = tnc.append(df_chunk[(df_chunk['Trip Start Timestamp']>=start_date)&(df_chunk['Trip Start Timestamp']<=end_date)])
<ipython-input-330-2a617723eaa4>:9: FutureWarning: Comparison

chicago_tnc_20190601_20190831.csv part


<ipython-input-330-2a617723eaa4>:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= e) & (start_date <= s):
<ipython-input-330-2a617723eaa4>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tnc = tnc.append(df_chunk[(df_chunk['Trip Start Timestamp']>=start_date)&(df_chunk['Trip Start Timestamp']<=end_date)])
<ipython-input-330-2a617723eaa4>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= s) | (start_date <= e):


chicago_tnc_20190901_20190930.csv part


<ipython-input-330-2a617723eaa4>:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= e) & (start_date <= s):
<ipython-input-330-2a617723eaa4>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tnc = tnc.append(df_chunk[(df_chunk['Trip Start Timestamp']>=start_date)&(df_chunk['Trip Start Timestamp']<=end_date)])
<ipython-input-330-2a617723eaa4>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= s) | (start_date <= e):


...

chicago_tnc_20211201_20211231.csv part


<ipython-input-330-2a617723eaa4>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= s) | (start_date <= e):
<ipython-input-330-2a617723eaa4>:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= e) & (start_date <= s):
<ipython-input-330-2a617723eaa4>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tnc = tnc.append(df_chunk[(df_chunk['Trip Start Timestamp']>=start_date)&(df_chunk['Trip Start Timestamp']<=end_date)])


chicago_tnc_20220101_20220131.csv part


<ipython-input-330-2a617723eaa4>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= s) | (start_date <= e):
<ipython-input-330-2a617723eaa4>:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if (end_date >= e) & (start_date <= s):
<ipython-input-330-2a617723eaa4>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tnc = tnc.append(df_chunk[(df_chunk['Trip Start Timestamp']>=start_date)&(df_chunk['Trip Start Timestamp']<=end_date)])


chicago_tnc_20220201_20220228.csv part


In [331]:
tnc['start_hour'] = tnc['Trip Start Timestamp'].dt.hour
tnc['start_min'] = tnc['Trip Start Timestamp'].dt.minute
tnc['quarter_hour'] = tnc['start_hour']*4 + tnc['start_min']//15
tnc['day'] = tnc['Trip Start Timestamp'].dt.date
tnc['day_index'] = tnc['Trip Start Timestamp'].dt.date.map(daydict)
tnc['ts'] = tnc['day_index']*96 + tnc['quarter_hour']

tnc['Pickup Census Tract'] = tnc['Pickup Census Tract'].astype(int).astype(str)

In [332]:
tnc_out = tnc.groupby(['ts','day','day_index','quarter_hour','Pickup Census Tract'], as_index=False).count()\
[['ts','day','day_index','quarter_hour','Pickup Census Tract','Fare']]

In [333]:
tnc_out.columns = ['ts','day','day_index','quarter_hour','GEOID10','tnc_count']

In [334]:
tnc_out['STATEFP10'] = tnc_out['GEOID10'].str[:2].astype(int)
tnc_out['COUNTYFP10'] = tnc_out['GEOID10'].str[2:5].astype(int)
tnc_out['TRACTCE10'] = tnc_out['GEOID10'].str[5:].astype(int)

tnc_out['GEOID10'] = tnc_out['STATEFP10'].astype(str)+"_"+tnc_out['COUNTYFP10'].astype(str)+"_"+tnc_out['TRACTCE10'].astype(str)

tnc_out = tnc_out[['ts','day','day_index','quarter_hour','GEOID10','tnc_count']]

In [335]:
tnc_out.to_csv(project_data_dir+'data_processed/census_tract/tnc_df_'+start_date.strftime("%y%m%d")+'_'+end_date.strftime("%y%m%d")+'.csv', index=False)

# Demographic Data
2018 demographic data's resolution is block groups


This block groups is different from the census tract 10 block groups!!

In [146]:
blkgrps = gpd.read_file(data_dir+'data_raw/tl_2018_17_bg/tl_2018_17_bg.shp')
blkgrps['COUNTYFP'] = blkgrps['COUNTYFP'].astype(int)
blkgrps['TRACTCE'] = blkgrps['TRACTCE'].astype(int)
blkgrps['BLKGRPCE'] = blkgrps['BLKGRPCE'].astype(int)

blkgrps = blkgrps.to_crs("epsg:26916")
blkgrps['area'] = blkgrps['geometry'].map(lambda p: p.area)

In [147]:
# demographic information of census blocks
demos = pd.read_csv(project_data_dir+'data_raw/nhgis0004_ds239_20185_2018_blck_grp.csv')

In [148]:
data_cols = [x for x in list(demos.columns) if len(x) == 8 and x[0:2]=='AJ' and x[4]=='E']
# This will tell us if we have any non-numeic data types
# print(demo_to_grid[data_cols].dtypes.value_counts())
# Run this to find out which columns are non-numeric. Then can feed them to the next line
# demo_to_grid[data_cols].select_dtypes(include=['int64']).columns
data_cols.remove('AJ0EE001')

In [149]:
travel_time_cols = ['AJXEE0'+str(i).zfill(2) for i in range(2,14)]
tt_medians = [2.5,7,12,17,22,27,32,37,42,52,74.5,100]

# aggregate values for each station.
demo_tract = demos.groupby(['STATEA','COUNTYA','TRACTA'])[data_cols].sum().reset_index()

In [150]:
# create demographics for each cell.
demo_tract['pct25_34yrs'] = (demo_tract[['AJWBE011','AJWBE012',
                                         'AJWBE035','AJWBE036']].sum(axis = 1))/demo_tract['AJWBE001']
demo_tract['pct35_50yrs'] = (demo_tract[['AJWBE013','AJWBE014','AJWBE015',
                                         'AJWBE037','AJWBE038','AJWBE039']].sum(axis = 1))/demo_tract['AJWBE001']
demo_tract['pctover65yrs'] = (demo_tract[['AJWBE020','AJWBE021','AJWBE022','AJWBE023','AJWBE024','AJWBE025',
                                          'AJWBE044','AJWBE045','AJWBE046','AJWBE047','AJWBE048','AJWBE049']].sum(axis = 1))/demo_tract['AJWBE001']
demo_tract['pctwhite_alone'] = demo_tract['AJWNE002']/demo_tract['AJWNE001']
demo_tract['pct_nonwhite'] = (demo_tract['AJWNE001']-demo_tract['AJWNE002'])/demo_tract['AJWNE001']
demo_tract['pctblack_alone'] = demo_tract['AJWNE003']/demo_tract['AJWNE001']
demo_tract['pct_hs_grad'] = (demo_tract[['AJYPE017','AJYPE018','AJYPE019','AJYPE020','AJYPE021','AJYPE022',
                                         'AJYPE023','AJYPE024','AJYPE025']].sum(axis = 1))/demo_tract['AJYPE001']
demo_tract['pct_col_grad'] = (demo_tract[['AJYPE022','AJYPE023','AJYPE024','AJYPE025']].sum(axis = 1))/demo_tract['AJYPE001']
demo_tract['pctPTcommute'] = demo_tract['AJXCE010']/demo_tract['AJXCE001']

demo_tract['25_34yrs'] = (demo_tract[['AJWBE011','AJWBE012',
                                         'AJWBE035','AJWBE036']].sum(axis = 1))
demo_tract['35_50yrs'] = (demo_tract[['AJWBE013','AJWBE014','AJWBE015',
                                         'AJWBE037','AJWBE038','AJWBE039']].sum(axis = 1))
demo_tract['over65yrs'] = (demo_tract[['AJWBE020','AJWBE021','AJWBE022','AJWBE023','AJWBE024','AJWBE025',
                                          'AJWBE044','AJWBE045','AJWBE046','AJWBE047','AJWBE048','AJWBE049']].sum(axis = 1))
demo_tract['white_alone'] = demo_tract['AJWNE002']/demo_tract['AJWNE001']
demo_tract['nonwhite'] = demo_tract['AJWNE001']-demo_tract['AJWNE002']
demo_tract['black_alone'] = demo_tract['AJWNE003']
demo_tract['hs_grad'] = (demo_tract[['AJYPE017','AJYPE018','AJYPE019','AJYPE020','AJYPE021','AJYPE022',
                                         'AJYPE023','AJYPE024','AJYPE025']].sum(axis = 1))
demo_tract['col_grad'] = (demo_tract[['AJYPE022','AJYPE023','AJYPE024','AJYPE025']].sum(axis = 1))
demo_tract['PTcommute'] = demo_tract['AJXCE010']

demo_tract['avg_tt_to_work'] = (demo_tract[travel_time_cols].multiply(tt_medians, axis = 1).sum(axis =1))/demo_tract['AJXEE001']

# create income info
# demo_tract['pct60plusmincommute'] = (demo_tract[['AJXEE012','AJXEE013']].sum(axis=1))/demo_tract['AJXEE001']
cell_inc = demos.groupby(['STATEA','COUNTYA','TRACTA']).apply(lambda x: 
                    (x['AJWME001']*x['AJ0EE001']).sum(axis = 0)/x['AJWME001'].sum(axis=0)).reset_index()
cell_inc.columns = ['STATEA','COUNTYA','TRACTA','inc_per_capita']
demo_tract = pd.merge(demo_tract, cell_inc, how = 'left', on = ['STATEA','COUNTYA','TRACTA'])

<ipython-input-150-f7aa229c6690>:35: RuntimeWarning: invalid value encountered in double_scalars
  (x['AJWME001']*x['AJ0EE001']).sum(axis = 0)/x['AJWME001'].sum(axis=0)).reset_index()


In [151]:
# extract demo data
demo_df = demo_tract[['STATEA','COUNTYA','TRACTA','AJWME001',
                       'pct25_34yrs','pct35_50yrs','pctover65yrs',
                       'pctwhite_alone','pct_nonwhite','pctblack_alone',
                       'pct_col_grad','pctPTcommute',
                       '25_34yrs','35_50yrs','over65yrs',
                       'white_alone','nonwhite','black_alone',
                       'col_grad','PTcommute',
                       'avg_tt_to_work','inc_per_capita']].fillna(0)

demo_df = demo_df.rename(columns = {'AJWME001':'tot_population'})

# demo_colnames = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
#          'pctwhite_alone','pct_nonwhite','pctblack_alone',
#          'pct_col_grad','pctPTcommute','25_34yrs','35_50yrs','over65yrs',
#          'white_alone','nonwhite','black_alone',
#          'col_grad','PTcommute','avg_tt_to_work','inc_per_capita','area']

demo_df['GEOID10'] = demo_df['STATEA'].astype(str)+"_"+demo_df['COUNTYA'].astype(str)+"_"+demo_df['TRACTA'].astype(str)

In [178]:
poi = gpd.read_file(project_data_dir+'data_raw/POI_OSM/gis_osm_pois_free_1.shp')

non_attractions = ['water_tower','toilet','post_box','pitch','drinking_water','atm','comms_tower','prison',
                   'recycling','camera_surveillance','windmill','bench','waste_basket','water_well','telephone',
                  'vending_machine','recycling_metal','wastewater_plant','reclycling_glass','chemist',
                  'recycling_paper','wayside_cross','recycling_clothes']

### Use category data from Patrick
poi_cat = pd.read_csv(project_data_dir+'data_raw/POIcat.csv')

poi_cat.columns = ['fclass','cat','start_active_time','end_active_time']

poi_map = pd.Series(poi_cat.cat.values,index=poi_cat.fclass).to_dict()

poi['category'] = poi['fclass'].map(poi_map).fillna('other')

### further aggregate
cat2 = {'bar':'restaurant','entertainment':'entertainment', 'hotel':'entertainment', 
        'public attraction':'entertainment', 'restaurant':'restaurant',
        'school':'school', 'services':'shop', 'shop':'shop'}
poi['category_2'] = poi['category'].map(cat2)
poi.dropna(inplace=True)

categories = list(poi['category_2'].unique())

poi = poi[~poi['fclass'].isin(non_attractions)]
poi.crs = CRS('epsg:4326')
poi = poi.to_crs("epsg:26916")

In [184]:
poi_census = gpd.sjoin(poi, tract_gdf, how="inner", op='intersects')
p = poi_census.groupby(['GEOID10','category_2']).size().unstack().reset_index().fillna(0)
p['all'] = p[categories].sum(axis = 1)

# save
demo_df = pd.merge(demo_df, p, on=['GEOID10'])

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [152]:
# job
jobs = pd.read_csv(project_data_dir+'data_raw/il_wac_S000_JT00_2017.csv', usecols = [0,1])
jobs.columns = ['GEOID10','jobs']

jobs['GEOID10'] = jobs['GEOID10'].astype(str)
jobs['STATEA'] = jobs['GEOID10'].str[:2].astype(int)
jobs['COUNTYA'] = jobs['GEOID10'].str[2:5].astype(int)
jobs['TRACTA'] = jobs['GEOID10'].str[5:11].astype(int)
jobs = jobs.groupby(['STATEA','COUNTYA','TRACTA'])['jobs'].sum().reset_index()

demo_df = pd.merge(demo_df, jobs, on=['STATEA','COUNTYA','TRACTA'])

In [186]:
# save demographics data
demo_df.to_csv(project_data_dir+"data_processed/census_tract/other/spatial.csv", index=False)

# Weather Data

In [154]:
weather = pd.read_csv(project_data_dir+'data_raw/weather.csv', parse_dates = ['DATE'])
weather = weather[weather['NAME']=='CHICAGO OHARE INTERNATIONAL AIRPORT, IL US']

weather.fillna(0, inplace=True)
weather.sort_values(by = 'DATE', inplace=True)
weather = weather[(weather['DATE']>=start_date)&(weather['DATE']<=end_date)]

weather['week'] = weather.DATE.dt.isocalendar().week
weather['year'] = weather.DATE.dt.isocalendar().year
weather['TAVGAVG'] = weather.groupby(['week','year']).transform(np.mean)['TAVG']

weather['PRCP'] = weather['PRCP'] + weather['SNOW']

weather['DAY_INDEX'] = weather['DATE'].map(daydict)
weather[['DATE','DAY_INDEX','PRCP','TAVGAVG','TAVG']].to_csv(project_data_dir+'data_processed/census_tract/weather_'+start_date.strftime("%y%m%d")+'_'+end_date.strftime("%y%m%d")+'.csv', index=False)


<ipython-input-154-aa0bd9ad9a59>:10: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  weather['TAVGAVG'] = weather.groupby(['week','year']).transform(np.mean)['TAVG']


# Level of Service: Bus Frequency

In [155]:
# Get GFTS Versions
gtfs_versions = [name.split("-") for name in os.listdir(project_data_dir+"data_raw/CTA_GTFS")]
gtfs_versions = pd.DataFrame(gtfs_versions, columns=['start_date','end_date'])
gtfs_versions['start_date'] = pd.to_datetime(gtfs_versions['start_date']).dt.date
gtfs_versions['end_date'] = pd.to_datetime(gtfs_versions['end_date']).dt.date
gtfs_versions = gtfs_versions.sort_values('end_date', ascending=False).reset_index(drop=True)

In [156]:
def get_service_id(path, cur_date):
    
    path = project_data_dir + "data_raw/CTA_GTFS/"+folder
#     s = pd.read_csv(path+'/stops.txt', dtype={'trip_id':str})
#     r = pd.read_csv(path+'/routes.txt') 
#     t = pd.read_csv(path+'/trips.txt', dtype={'service_id':str, 'trip_id':str, 'shape_id':str, 'schd_trip_id':str})
#     st = pd.read_csv(path+'/stop_times.txt', dtype={'trip_id':str})
    c = pd.read_csv(path+'/calendar.txt', dtype={'service_id':str, 'start_date':str, 'end_date':str})
    cd = pd.read_csv(path+'/calendar_dates.txt', dtype={'service_id':str, 'date':str})

    c['start_date'] = pd.to_datetime(c['start_date'])
    c['end_date'] = pd.to_datetime(c['end_date'])
    cd['date'] = pd.to_datetime(cd['date'])

    # get service_ids from calendar.txt
    dow_col = cur_date.day_name().lower()
    service_ids = c[(c[dow_col] == 1) & (c['start_date']<=cur_date) & (c['end_date']>=cur_date)]['service_id']

    # modify according to calendar_dates.txt
    service_ids = pd.concat([service_ids, cd[(cd['date']==cur_date) & (cd['exception_type']==1)]['service_id']])
    service_ids = service_ids[~service_ids.isin(cd[(cd['date']==cur_date) & (cd['exception_type']==2)]['service_id'])]

    return service_ids.to_list()

In [157]:
def get_counts_from_gtfs(path, cur_date):
    
    path = project_data_dir + "data_raw/CTA_GTFS/"+folder
    s = pd.read_csv(path+'/stops.txt', dtype={'trip_id':str})
    r = pd.read_csv(path+'/routes.txt') 
    t = pd.read_csv(path+'/trips.txt', dtype={'service_id':str, 'trip_id':str, 'shape_id':str, 'schd_trip_id':str})
    st = pd.read_csv(path+'/stop_times.txt', dtype={'trip_id':str})
    c = pd.read_csv(path+'/calendar.txt', dtype={'service_id':str, 'start_date':str, 'end_date':str})
    cd = pd.read_csv(path+'/calendar_dates.txt', dtype={'service_id':str, 'date':str})

    c['start_date'] = pd.to_datetime(c['start_date'])
    c['end_date'] = pd.to_datetime(c['end_date'])
    cd['date'] = pd.to_datetime(cd['date'])

    # get service_ids from calendar.txt
    dow_col = cur_date.day_name().lower()
    service_ids = c[(c[dow_col] == 1) & (c['start_date']<=cur_date) & (c['end_date']>=cur_date)]['service_id']

    # modify according to calendar_dates.txt
    service_ids = pd.concat([service_ids, cd[(cd['date']==cur_date) & (cd['exception_type']==1)]['service_id']])
    service_ids = service_ids[~service_ids.isin(cd[(cd['date']==cur_date) & (cd['exception_type']==2)]['service_id'])]

    # get ids of scheduled trips from trips.txt
    schd_trips = t[t['service_id'].isin(service_ids.tolist())]

    # filter to bus routes
    r = r[r['route_type']==3]
   
    # count stop times for identified trips and stops
    stt = st.merge(s, on='stop_id')
    stt = stt.merge(schd_trips, how = 'inner', on = 'trip_id')
    stt = stt.merge(r, how = 'inner', on = 'route_id')
    stt['arrival_time'] = stt['arrival_time'].astype(str)
    stt['arrival_hour'] = stt['arrival_time'].str[:2].astype(int)
    stt['arrival_min'] = stt['arrival_time'].str[3:5].astype(int)
    stt['quarter_hour'] = stt['arrival_hour'] * 4 + stt['arrival_min']//15
    stt['ts'] = ts_day_base + stt['quarter_hour']
    
    # spatial join with census tracts
    point_geom = [Point(xy) for xy in zip(stt['stop_lon'],stt['stop_lat'])]
    stt = gpd.GeoDataFrame(stt, crs=CRS("epsg:4326"), geometry=point_geom)
    stt = stt.to_crs(CRS("epsg:26916"))    
    
    stt = gpd.sjoin(tract_gdf, stt, how='left', op='contains')
 
    # 
    counts = stt.groupby(['ts','quarter_hour','GEOID10'], as_index=False).count()\
                    [['ts','quarter_hour','GEOID10','trip_id']]
    
    return counts

In [158]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)

ts_day_base = 0
bus_freq = pd.DataFrame([])    
for cur_date in daterange(start_date, end_date):
    day_index = daydict[cur_date.date()]
    ver = gtfs_versions[(cur_date >= pd.to_datetime(gtfs_versions['start_date'])) \
                        & (cur_date <= pd.to_datetime(gtfs_versions['end_date']))]
    print(day_index, cur_date, end='\t')
    if len(ver)==0:
        ts_day_base += 96
        print(cur_date, 'not found!')
        continue
    else:
        folder = datetime.datetime.strftime(gtfs_versions['start_date'].iloc[ver.index[0]],"%Y%m%d") + '-' + \
                datetime.datetime.strftime(gtfs_versions['end_date'].iloc[ver.index[0]],"%Y%m%d")
        new = get_counts_from_gtfs(folder, cur_date)
        
        new['day_index'] = day_index
        new['day'] = cur_date
        bus_freq = pd.concat([bus_freq, new])
        ts_day_base += 96


0 2020-03-02 00:00:00	

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


1 2020-03-03 00:00:00	

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


2 2020-03-04 00:00:00	

...

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


241 2020-10-29 00:00:00	

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


242 2020-10-30 00:00:00	

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
bus_freq.columns = ['ts','quarter_hour','GEOID10','num_schd_trp_15min', 'day_index', 'day']
bus_freq.to_csv(project_data_dir+'data_processed/census_tract/los_df_'+start_date.strftime("%y%m%d")+'_'+end_date.strftime("%y%m%d")+'.csv', index=False)